In [ ]:
import networkx as nx
import numpy as np
import MDAnalysis as mda
import random
import warnings
from MDAnalysis.analysis import distances, contacts
import ast,os
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [ ]:
def get_chain_length(alpha_carbons: mda.AtomGroup) -> int:
    """
    Calculate the chain length of protein from alpha carbons.
    @param alpha_carbons
    @return: length of chain
    """
    chain_length = 0
    if alpha_carbons:
        chain_length = len(alpha_carbons)
    return chain_length


def get_distance_matrix(alpha_carbons: mda.AtomGroup, distance_array: np.ndarray) -> np.ndarray:
    """
    Get distance matrix for PDBs from distance array and C-alphas
    @param: alpha_carbons C-alphas from PDB
    @param: distance_array distance array from C-alphas
    @return: distance_matrix as a numpy array
    """
    n_alpha_carbons = get_chain_length(alpha_carbons)
    distance_matrix = np.zeros((n_alpha_carbons, n_alpha_carbons))
    n_rows = distance_matrix.shape[0]
    diagonal = 1
    upper_triangle = np.triu_indices(n_rows, k=diagonal)
    distance_matrix[upper_triangle] = distance_array
    np.fill_diagonal(distance_matrix, 0)
    for i in range(len(distance_matrix)):
        for j in range(i):
            distance_matrix[i, j] = distance_matrix[j, i]
    return distance_matrix


def get_distance_array(alpha_carbons: mda.AtomGroup) -> np.ndarray:
    """
    Use the C-alphas to get the distance array
    @return: distance array
    """
    return distances.self_distance_array(alpha_carbons.positions)


In [ ]:
class ProteinContactMap:
    """
    Base class for ProteinContactMaps.
    """

    def __init__(self, pdb_file, default_threshold=8.0):
        """
        Construct the ProteinContactMap and initialise default threshold.
        @param pdb_file: str
        @param default_threshold: float
        """
        self.universe = mda.Universe(pdb_file)
        self.threshold = default_threshold

    @property
    def threshold(self):
        """
        Get threshold
        @return:
        """
        return self._threshold

    @threshold.setter
    def threshold(self, threshold_value: float) -> None:
        """
        Set threshold value
        @param threshold_value:
        @return: None
        """
        if threshold_value != float(threshold_value):
            raise TypeError('The threshold value must be a float.')

        if threshold_value >= 0:
            self._threshold = threshold_value

        else:
            raise ValueError('The threshold value must be larger than 0.')

    @threshold.deleter
    def threshold(self) -> None:
        """
        Deleter for threshold
        @return: None
        """
        raise AttributeError("The threshold value cannot be deleted. You can set it to 0.")

    @property
    def get_alpha_carbons(self) -> None:
        """
        Take a universe, get segment A and return the C-alphas for the PDB. 

        Parameters
        ----------
        universe: MDAnalysis.Universe(PDB_file)

        Return
        ------
        C_alphas: MDAnalysis AtomGroup
            contains the C-alphas from the PDB
        """
        segments = self.universe.residues.segments
        n_segments = len(segments)
        alpha_carbons = None
        for i in range(n_segments):
            segment_id = self.universe.residues.segments.segids[i]
            try:
                protein_atoms = self.universe.select_atoms('protein')
                mda_altlocs = protein_atoms.altLocs
                alternative_locations = []
                for loc in mda_altlocs:
                    if loc != '':
                        alternative_locations.append(loc)
                alternative_locations = np.sort(list(set(alternative_locations)))
                n_altlocs = len(alternative_locations)
                if n_altlocs > 1:
                    residue_ids = protein_atoms.residues.resids
                    alpha_carbons = self.universe.select_atoms(
                        f"resid {residue_ids[0]}:{residue_ids[-1]} and name CA and segid {segment_id}")
                    for j in range(1, n_altlocs):
                        exclude_atoms = self.universe.select_atoms(
                            f"resid {residue_ids[0]}:{residue_ids[-1]} and name CA and segid {segment_id} and altLoc "
                            f"{alternative_locations[j]}")
                        alpha_carbons -= exclude_atoms
                else:
                    residue_ids = protein_atoms.residues.resids
                    alpha_carbons = self.universe.select_atoms(
                        f"resid {residue_ids[0]}:{residue_ids[-1]} and name CA and segid {segment_id}")
                break

            except mda.SelectionError:
                print("Error in first segment id. Trying the next one.")
                continue
        return alpha_carbons

    def get_contacts_array(self, distance_array: np.ndarray) -> np.ndarray:
        """
        Use the distance array to get contacts array
        @return: contacts array
        """
        return contacts.contact_matrix(distance_array, radius=self.threshold)

    def get_cmap(self, alpha_carbons: mda.AtomGroup, distance_array: np.ndarray) -> np.ndarray:
        """
        Get contact map from PDB
        @param distance_array: distance array from C-alphas
        @param alpha_carbons: C-alphas from PDB
        @return: adjacency matrix as numpy array
        """
        contacts_array = self.get_contacts_array(distance_array)
        adjacency_array = np.zeros(len(contacts_array))
        for i in range(len(contacts_array)):
            if contacts_array[i]:
                adjacency_array[i] = 1
        n_alpha_carbons = get_chain_length(alpha_carbons)
        pcm_matrix = np.zeros((n_alpha_carbons, n_alpha_carbons))
        n_rows = pcm_matrix.shape[0]
        diagonal = 1
        upper_triangle = np.triu_indices(n_rows, k=diagonal)
        pcm_matrix[upper_triangle] = adjacency_array
        np.fill_diagonal(pcm_matrix, 1)
        return np.where(pcm_matrix, pcm_matrix, pcm_matrix.T)



In [ ]:
def pdb_to_cmap(pdb_file: str) -> tuple:
    """
    Convert given PDB file to a contact map
    @param pdb_file: PDB file from RCSB or AlphaFold in each length range
    @return: tuple of the adjacency and distance matrices as numpy arrays
    """
    pcm = ProteinContactMap(pdb_file, default_threshold=8.0)
    alpha_carbons = pcm.get_alpha_carbons
    distance_array = get_distance_array(alpha_carbons)
    distance_matrix = get_distance_matrix(alpha_carbons, distance_array)
    cmap = pcm.get_cmap(alpha_carbons, distance_array)
    return distance_matrix, cmap

In [ ]:
file=open("/data/davis/proteins_updated.txt","r")
contents=file.read()
protein_data=ast.literal_eval(contents)
protein_data.keys()
list_prots=list(protein_data.keys())
len(list_prots)

In [ ]:
def main():
    PDB_IDs = list_prots
    #PDB_IDs = ['P80192']
    results_path="/data/davis/alpha_fold_cmaps/" 
    for i,j in enumerate(PDB_IDs):
        print(PDB_IDs[i])
        PDB_IDs_lower = str(PDB_IDs[i])
        #Set path to PDB files
        PDB_file = f'//alphafold2_new_pdb/{PDB_IDs_lower}.pdb' 
        if os.path.isfile(PDB_file):
            pcm = ProteinContactMap(PDB_file, default_threshold=8.0)
            alpha_carbons = pcm.get_alpha_carbons
            distance_array = get_distance_array(alpha_carbons)
            distance_matrix = get_distance_matrix(alpha_carbons, distance_array)
            cmap = pcm.get_cmap(alpha_carbons, distance_array)
            plt.imshow(cmap)
            print(cmap.shape)
            savepath='/data/davis/alpha_fold_cmaps/'+PDB_IDs_lower+'.npy'
            np.save(savepath,cmap)

In [ ]:
main()